In [1]:
using Pkg; Pkg.activate("..")
using Graphs, GraphPlot, Compose, MetaGraphsNext

  Activating project at `~/study/7CCSMBIM_Algorithms`


In [2]:
g = MetaGraph(Graph(), EdgeData = Int, graph_data = "TSP", weight_function = identity, default_weight = -1);
# graph vertices
g[:A] = nothing;
g[:B] = nothing;
g[:C] = nothing;
g[:D] = nothing;
g[:E] = nothing;
g[:F] = nothing;
g[:G] = nothing;
g[:H] = nothing;
# graph edges
g[:A, :B] = 3;
g[:A, :D] = 5;
g[:B, :C] = 5;
g[:B, :E] = 4;
g[:B, :F] = 8;
g[:C, :D] = 1;
g[:C, :G] = 4;
g[:C, :H] = 7;
g[:C, :E] = 2;
g[:D, :G] = 2;
g[:D, :H] = 6;
g[:E, :F] = 10;
g[:E, :H] = 3;
g[:G, :H] = 2;
collect(weights(g))

8×8 Matrix{Int64}:
 -1   3  -1   5  -1  -1  -1  -1
  3  -1   5  -1   4   8  -1  -1
 -1   5  -1   1   2  -1   4   7
  5  -1   1  -1  -1  -1   2   6
 -1   4   2  -1  -1  10  -1   3
 -1   8  -1  -1  10  -1  -1  -1
 -1  -1   4   2  -1  -1  -1   2
 -1  -1   7   6   3  -1   2  -1

# Prim Algorithm

In [181]:
argmin([5, 5, 1])

3

In [180]:
findmin([(1, 2), (5, 5)])

((1, 2), 1)

In [202]:
function prim_mst_algo(g, initial_vertex)
    mst = MetaGraph(Graph(), EdgeData=Int, graph_data="MST", weight_function=identity, default_weight=0);
    for v = vertices(g)
        mst[label_for(g, v)] = g[label_for(g, v)];
    end
    in_mst = [];
    seen = [code_for(g, initial_vertex)];
    
    while !issetequal(vertices(g), in_mst)
        nb = [n for n = neighbors(g, seen[end]) if !(n in seen)];
        if isempty(nb)
            throw("No solution!");
        end
        min_weight = nb[argmin([g[label_for(g, seen[end]), label_for(g, n)] for n in nb])];
        mst[label_for(g, seen[end]), label_for(g, min_weight)] = g[label_for(g, seen[end]), label_for(g, min_weight)];
        push!(in_mst, seen[end], min_weight);
        push!(seen, min_weight);
    end
    return mst, sum(weights(mst)) / 2
end

prim_mst_algo (generic function with 1 method)

In [203]:
mst = nothing;
cost = nothing;
for i = collect(vertices(g))
    try    
        mst, cost = prim_mst_algo(g, label_for(g, i));
    catch
        nothing;
    end
end
mst, cost

(Meta graph based on a SimpleGraph{Int64}(7, [[2, 4], [1, 6], [4, 5], [1, 3], [3, 8], [2], [8], [5, 7]]) with vertex labels of type Symbol, vertex metadata of type Nothing, edge metadata of type Int64, graph metadata given by "MST", and default weight 0, 24.0)

In [205]:
collect(edges(mst))

7-element Vector{Graphs.SimpleGraphs.SimpleEdge{Int64}}:
 Edge 1 => 2
 Edge 1 => 4
 Edge 2 => 6
 Edge 3 => 4
 Edge 3 => 5
 Edge 5 => 8
 Edge 7 => 8

In [208]:
for i = vertices(mst)
    println(i, ": ", label_for(mst, i))
end

1: A
2: B
3: C
4: D
5: E
6: F
7: G
8: H


In [163]:
a = prim_mst(g, weights(g))

7-element Vector{Graphs.SimpleGraphs.SimpleEdge{Int64}}:
 Edge 1 => 2
 Edge 5 => 3
 Edge 3 => 4
 Edge 2 => 5
 Edge 2 => 6
 Edge 4 => 7
 Edge 7 => 8

# Kruskal Algorithm

In [209]:
function kruskal_mst(g)
    wm = sort([(g[label_for(g, e.src), label_for(g, e.dst)], e.src, e.dst) for e in edges(g)]);
    mst = MetaGraph(Graph(), EdgeData=Int, graph_data="MST", weight_function=identity, default_weight=0);
    for v in vertices(g)
        mst[label_for(g, v)] = g[label_for(g, v)];
    end
    
    for (w, src, dst) = wm
        if ne(mst) == nv(mst)-1
            break
        end
        src_lb = label_for(g, src);
        dst_lb = label_for(g, dst);
        mst[src_lb, dst_lb] = g[src_lb, dst_lb];
        if is_cyclic(mst)
            rem_edge!(mst, src, dst);
        end
    end
    return mst, sum(weights(mst)) / 2
end

kruskal_mst (generic function with 1 method)

In [210]:
mst, cost = kruskal_mst(g)

(Meta graph based on a SimpleGraph{Int64}(7, [[2], [1, 5, 6], [4, 5], [3, 7], [2, 3], [2], [4, 8], [7]]) with vertex labels of type Symbol, vertex metadata of type Nothing, edge metadata of type Int64, graph metadata given by "MST", and default weight 0, 22.0)

In [211]:
collect(edges(mst))

7-element Vector{Graphs.SimpleGraphs.SimpleEdge{Int64}}:
 Edge 1 => 2
 Edge 2 => 5
 Edge 2 => 6
 Edge 3 => 4
 Edge 3 => 5
 Edge 4 => 7
 Edge 7 => 8

In [212]:
for v = vertices(mst)
    println(v, ": ", label_for(mst, v))
end

1: A
2: B
3: C
4: D
5: E
6: F
7: G
8: H
